In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import numpy as np
# Para la visualización 
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Otros objetivos
# -----------------------------------------------------------------------
import math
from itertools import combinations
# Para pruebas estadísticas
# -----------------------------------------------------------------------
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
# Para la codificación de las variables numéricas
# -----------------------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder # para poder aplicar los métodos de OneHot, Ordinal,  Label y Target Encoder 
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder, CountEncoder
import sys
sys.path.append("../../")
import src.soporte_encoding  as se
from src.soporte_graficas import (
    visualizar_categoricas
)
from src.SupportPreProcesamiento import (
    separarar_df
)

In [2]:
df = pd.read_csv("../../datos/03_api_rent_sin_nulos.csv")

# Encoding
- Vamos a pasar todas las variables a numéricas
- Hay que saber si tienen orden o no

### Pasar las columnas a su tipo correcto

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   propertyType  353 non-null    object 
 1   price         353 non-null    float64
 2   size          353 non-null    float64
 3   rooms         353 non-null    object 
 4   bathrooms     353 non-null    object 
 5   floor         353 non-null    object 
 6   municipality  353 non-null    object 
 7   exterior      353 non-null    object 
 8   hasLift       353 non-null    object 
 9   hasPlan       353 non-null    object 
 10  has360        353 non-null    object 
dtypes: float64(2), object(9)
memory usage: 30.5+ KB


In [4]:
df["rooms"] = df["rooms"].astype("O")
df["bathrooms"] = df["bathrooms"].astype("O")

# Analicemos estadísticamente
- Dos opciones: mannwhitneyu
- Mas de dos opciones: Kruskal

No es una distribución normal

In [5]:
numericas,categoricas = separarar_df(df)

In [6]:
# instanciamos la clase de Asunciones, para poder decir qué método usar para el análisis estadístico
asunciones = se.Asunciones(dataframe = df, columna_numerica = "price")
print("\n-------------\n")


for categoria in categoricas:   

    print(f"Estamos analizando la variable {categoria.upper()}")
    
    asunciones.identificar_normalidad(metodo = "kolmogorov") 

    # comprobamos la homogeneidad de varianzas
    asunciones.identificar_homogeneidad(columna_categorica = categoria)

    # instanciamos la clase para evaluar si hay diferencias entre los distintos grupos de las variables categóricas
    test = se.TestEstadisticos(df, "price", categoria)
    test.run_all_tests()
    print("\n###########################\n")


-------------

Estamos analizando la variable PROPERTYTYPE
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable propertyType las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['flat', 'penthouse', 'studio', 'duplex', 'chalet', 'countryHouse']

Realizando test de Kruskal...
Estadístico de prueba: 10.491276765664804
Valor p: 0.062453362538402654
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,-10.0000,1.0000,-207.3147,187.3147,chalet-countryHouse
1,79.2857,0.2949,-29.4294,188.0008,chalet-duplex
2,87.5596,0.1184,-11.6306,186.7497,chalet-flat
3,9.9000,0.9999,-102.5867,122.3867,chalet-penthouse
4,79.6250,0.2206,-22.0687,181.3187,chalet-studio
5,89.2857,0.6984,-87.5915,266.1629,countryHouse-duplex
6,97.5596,0.5771,-73.6282,268.7473,countryHouse-flat
7,19.9000,0.9996,-159.3200,199.1200,countryHouse-penthouse
8,89.6250,0.6724,-83.0254,262.2754,countryHouse-studio
9,8.2739,0.9959,-38.5355,55.0832,duplex-flat



###########################

Estamos analizando la variable ROOMS
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable rooms las varianzas son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['1 habitacion', '2 habitaciones', '3 habitaciones', 'sin habitaciones', '4 habitaciones']

Realizando test de Kruskal...
Estadístico de prueba: 6.997445029790361
Valor p: 0.13602330549397834
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,15.3159,0.2789,-5.9159,36.5476,1 habitacion-2 habitaciones
1,22.5779,0.1712,-5.1881,50.3438,1 habitacion-3 habitaciones
2,30.6804,0.9113,-67.1878,128.5487,1 habitacion-4 habitaciones
3,8.2043,0.9324,-20.1500,36.5585,1 habitacion-sin habitaciones
4,7.2620,0.9511,-20.2748,34.7988,2 habitaciones-3 habitaciones
5,15.3646,0.9928,-82.4389,113.1680,2 habitaciones-4 habitaciones
6,-7.1116,0.9579,-35.2415,21.0183,2 habitaciones-sin habitaciones
7,8.1026,0.9994,-91.3242,107.5293,3 habitaciones-4 habitaciones
8,-14.3736,0.7616,-47.7121,18.9648,3 habitaciones-sin habitaciones
9,-22.4762,0.9720,-122.0688,77.1164,4 habitaciones-sin habitaciones



###########################

Estamos analizando la variable BATHROOMS
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable bathrooms las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['1 aseo', '2 aseos', '3 aseos']

Realizando test de Kruskal...
Estadístico de prueba: 2.17914787981004
Valor p: 0.3363597726585303
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



...


###########################

Estamos analizando la variable HASPLAN
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable hasPlan las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['no tiene planos', 'tiene planos', 'desconocido']

Realizando test de Kruskal...
Estadístico de prueba: 7.183821049179696
Valor p: 0.027545653470666565
Hay una diferencia significativa entre los grupos
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,25.9515,0.2680,-13.3999,65.3029,desconocido-no tiene planos
1,42.6477,0.0355,2.2861,83.0093,desconocido-tiene planos
2,16.6962,0.0429,0.4164,32.9760,no tiene planos-tiene planos



###########################

Estamos analizando la variable HAS360
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable has360 las varianzas son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['no tiene fotos 360', 'tiene fotos 360', 'desconocido']

Realizando test de Kruskal...
Estadístico de prueba: 5.752988674847895
Valor p: 0.056331897713708286
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,29.0220,0.1943,-10.3926,68.4366,desconocido-no tiene fotos 360
1,39.8272,0.0620,-1.5453,81.1998,desconocido-tiene fotos 360
2,10.8052,0.3320,-7.1139,28.7243,no tiene fotos 360-tiene fotos 360



###########################



# Resultados
### Variables ordinales
- floor
- municipality
- hasLift
- hasPlan
### Variables Nominales
- propertyType
- rooms
- bathrooms
- exterior
- has360

# Tenemos Ordinales y Nominales
- Las ordinales las gestionaremos con Target Encoding, que codifica con la media del objetivo para la categoría
- Para las nominales usaremos one-Hot Encoding

In [7]:
df.sample()

,propertyType,price,size,rooms,bathrooms,floor,municipality,exterior,hasLift,hasPlan,has360
272,flat,750.0,42.0,1 habitacion,1 aseo,cuarto,Clase Media-Alta,vista interior,tiene ascensor,no tiene planos,no tiene fotos 360


# Ordinales

In [8]:
cols_ordinales = ["floor","municipality","hasLift","hasPlan"]

### Target Encoder

In [9]:
target_encoder = TargetEncoder(cols=cols_ordinales)
df = target_encoder.fit_transform(df,df[["price"]])

In [10]:
df.sample()

,propertyType,price,size,rooms,bathrooms,floor,municipality,exterior,hasLift,hasPlan,has360
324,flat,750.0,110.0,4 habitaciones,2 aseos,701.698078,702.071795,vista exterior,687.633544,688.380091,no tiene fotos 360


# Nominales

In [11]:
cols_nominales = ["propertyType","rooms","bathrooms","exterior","has360"]

### OneHotEncoder

In [12]:
encoder = OneHotEncoder(categories='auto', 
                        drop=None, 
                        sparse_output=True, 
                        dtype='float', 
                        handle_unknown='error')

# Ajustar el codificador a los datos y transformarlos
encoder_trans = encoder.fit_transform(df[cols_nominales])

# lo siguiente que hacemos es convertir el objeto devuelto por el fit_transform a array para poder verlo
encoder_array = encoder_trans.toarray()

# usaremos el método '.get_feature_names_out()' para extraer el nombre de las columnas
nombre_columnas = encoder.get_feature_names_out()

# creamos un DataFrame con los resultados obtenidos de la transformación
encoder_df = pd.DataFrame(encoder_array, columns = nombre_columnas)

# concatenamos estos resultados con el DataFrame original
df = pd.concat([df, encoder_df], axis = 1)

In [13]:
df.sample()

,propertyType,price,size,rooms,bathrooms,floor,municipality,exterior,hasLift,hasPlan,has360,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,rooms_sin habitaciones,bathrooms_1 aseo,bathrooms_2 aseos,bathrooms_3 aseos,exterior_vista exterior,exterior_vista interior,has360_desconocido,has360_no tiene fotos 360,has360_tiene fotos 360
343,flat,750.0,55.0,2 habitaciones,1 aseo,701.698078,702.071795,vista exterior,687.633544,688.380091,no tiene fotos 360,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


# Eliminar columnas nominales
- Ya tenemos las columnas generadas y ya no nos aportan información

In [14]:
df.drop(columns=cols_nominales,inplace=True)

In [15]:
df.sample()

,price,size,floor,municipality,hasLift,hasPlan,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,rooms_sin habitaciones,bathrooms_1 aseo,bathrooms_2 aseos,bathrooms_3 aseos,exterior_vista exterior,exterior_vista interior,has360_desconocido,has360_no tiene fotos 360,has360_tiene fotos 360
81,725.0,69.0,701.698078,677.821984,702.252872,688.380091,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


# Guardamos
- Nos vamos a feature scaling con este df

In [16]:
df.to_csv("../../datos/04_rent_target_onehot_encoded.csv",index=False)